## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

<figure>
 <img src="write_up_sources/test0.jpg" width="1280" alt="Combined Image" />
</figure>

## Compute the camera calibration using chessboard images

Function **cv2.calibrateCamera** finds the camera intrinsic and extrinsic parameters from several views of a calibration pattern.
We need distortion coefficients and camera matrix. As an input we provide 3d points in real world space and corresponding 2d points in image plane.<br>
The camera calibration pipeline can be found in *Project/camera_calibration.py*.
To execute camera calibration set **recalibrate** parameters to True and Run.

In [ ]:
# perform camera calibration
[ret, mtx, dist, rvecs, tvecs] = cv2.calibrateCamera(objpoints, imgpoints, (img.shape[1], img.shape[0]),None,None)
# Undistort image
img_undist = cv2.undistort(distorted_img, mtx, dist, None, mtx)

<figure>
 <img src="write_up_sources/undist.png" width="1280" alt="Combined Image" />
</figure>
 <p></p> 
<figure>
 <img src="write_up_sources/undist_real.png" width="1280" alt="Combined Image" />
</figure>

## Color spaces and gradient

Next step is to convert undistorted image to binary by applying absolute sobel edge threshold and magnitude thresholds. <br>
Input image is converted into **grayscale** and into **LAB** colour space. From LAB colour space dimension containing **L** channel is extracted. <br>
Result of grayscale are concatenated using **and** operator. Same for S channel. <br>
And two final results united into one using **or** operator.

### Tune thresholds

To find appropriate thresholds values the following interactive GUI was implemented:

<figure>
 <img src="write_up_sources/tune_thresh.gif" width="250%" alt="Combined Image"  />
</figure>

In [ ]:
# 0. Get L (LAB) channel from input image
    l_channel_img = get_L_channel(input_img)

    # 1. Calculate absolute threshold and magnitude threshold for grayscale image
    abs_sobel_bin_img = abs_sobel_thresh(input_img, orient='x', thresh=(20, 100))
    mag_thresh_bin_img = mag_thresh(input_img, sobel_kernel=5, mag_thresh=(20, 100))

    # 2.  Calculate absolute threshold and magnitude threshold for s channel image
    abs_sobel_bin_img_gray = abs_sobel_thresh(l_channel_img, orient='x', thresh=(50, 100))
    mag_thresh_bin_img_gray = mag_thresh(l_channel_img, sobel_kernel=15, mag_thresh=(30, 100))

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(s_channel_img)
    combined_binary[((abs_sobel_bin_img == 1) & (mag_thresh_bin_img == 1))
                    | ((abs_sobel_bin_img_gray == 1) | (mag_thresh_bin_img_gray == 1))] = 1

<figure>
 <img src="write_up_sources/color_and_mag_thresh.png" width="250%" alt="Combined Image"  />
</figure>

To play around with thresholds value is possible in *Project/combine_thresholds.py*. Parameters can be changes in **convert_and_threshold** function. And to see results set in **main** function for **convert_and_threshold(img, visu = False)** call **visu** parameter to True. Results are written into *test_images/combined_binary* folder and are used in next step - *Project/get_perspective.py*.

## Perspective transformation

Next step is to transform image top-down way. It will allow us to measure the curvation of lanes. And also we can focus only on region where lane lines are present.<br>
In *Project/get_perspective.py* is pipeline for tuning prespective transformation matrix.<br>

First, we need to select source points on an image:

<figure>
 <img src=write_up_sources/choose_points_persp.png" width="250%" alt="Combined Image"  />
</figure>

Points shall be chosen in following order:
1. upper left
2. upper right
3. bottom right
4. bottom left <br>
Press **'q'** to quit. <br>
Drawn lines are intended just to visually help to the user to choose good source points. <br>
From the observation if lines are strongly curved, then choose lines which form 'low' trapezium. In case of almost parallel lines its good to form 'high' trapezium with narrow upper edge. <br>
On the next image it is required to choosen the region of interest. Points shall be chosen in the same order as in the previous step. The rest of the image is set to zero.

<figure>
 <img src="write_up_sources/choose_points_persp_res.png" width="250%" alt="Combined Image"  />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Strongly curved lines </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="write_up_sources/choose_points_persp_res3.png" width="250%" alt="Combined Image"  />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Almost straight lines </p> 
 </figcaption>
</figure>

This is why important to do the thresholding first and then perform tranformation:

<figure>
 <img src="write_up_sources/lost_info_on_color.png" alt="Combined Image"  />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> The information about left line is almost lost </p> 
 </figcaption>
</figure>
 <p></p> 
 <figure>
 <img src="write_up_sources/lost_info_on_color_vs_binary.png" alt="Combined Image"  />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> More information about left line </p> 
 </figcaption>
</figure>

Result are stored into *Project/test_images/warped* folder. To visualize stored results set **show_results** to True in **main** function.

## Locate the Lane Lines using histogram

Now we are moving into *Project/find_lane_pixels.py* file. First we check results from previous step by taking the histogram along all the columns in the lower half of the image. This is the part of the image where lanes supposed to be almost parallel to the vehicle's moving direction. <br>
You can uncomment **test_histogram** function in the **main** and see histograms visualization of all wrapped images generated in the previous step.

In [23]:
histogram = np.sum(img[height//2:height,:], axis=0)

<figure>
 <img src="write_up_sources/histogram_visu.png" alt="Combined Image"  />
</figure>
 <p></p> 
<figure>
 <img src="write_up_sources/histogram_visu_2.png" alt="Combined Image"  />
</figure>

## Find histogram picks

**find_lane_pixels_test** function implements a pipeline for detecting lanes and calculating the polynomial describing them.
As a first step in this function is locating the maximum value in histogram for the left and right halfs.

In [ ]:
# Find the peak of the left and right halves of the histogram
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

Calculated peaks are used as a starting point for where to search for the lines. Then using sliding windows approach we look for all indices which are not zero inside these windows for left and right lanes. The center position of the window is corrected in each step by computing the arithmetic mean (average) of the given data along the x axe.

In [40]:
# update center position for the next sliding window
if len(good_left_inds) > minpix:
    leftx_current = np.int(np.mean(nonzerox[good_left_inds]))

<figure>
 <img src="write_up_sources/sliding_window_1.png" alt="Combined Image"  />
</figure>
 <p></p> 

Next step is to fit a second order polynomial to each using `np.polyfit`

In [ ]:
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

Resulting left and right pixels are used to draw a polynomial on an image. This image serves as an input for inverse perspective transformation. Result is combined with original image.

<figure>
 <img src="write_up_sources/result_0.png" alt="Combined Image"  />
</figure>
 <p></p>
<figure>
 <img src="write_up_sources/result_2.png" alt="Combined Image"  />
</figure>

There is also a possibility to speed up calculations providing as an input into **find_good_indexis_in_window** not all indices of the elements that are non-zero, but only the one which were detected as right and left lanes in the previous frame. However there is a risk that in case there is large curve on the road ahead we can easily lost a track. In that case we will have to come back one step back and search for sliding window center again using all non-zero pixels.

### Another Sliding Window Search Approach - convolution

There is also an option to convolve each horizontal slice of the image with a window mask. The maximums of convolution result for the left and right halfs of the window correspond to center of windows which define left and right lanes respectively. This approach can be handy in case of large curved on the road. To test it run **main** in *Project/find_lane_pixels_convolution.py*. Also it's possible to visually compare results of "histogram" and "convolution" approaches by uncommenting **compare_with_hist_approach** function.

<figure>
 <img src="write_up_sources/convolve_approach.png" width=65% alt="Combined Image"  />
</figure>
 <p></p>
<figure>
 <img src="write_up_sources/hist_vs_conv_0.png" width=60% alt="Combined Image"  />
</figure>
 <p></p>
<figure>
 <img src="write_up_sources/hist_vs_conv_1.png" width=60% alt="Combined Image"  />
</figure>
 <p></p>
<figure>
 <img src="write_up_sources/result_convolution.png" width=200% alt="Combined Image"  />
</figure>

## Calculate curvation radius

Next we can calculate curvation of an image using parameters of the polynomial we received from **np.polyfit** function.

$$R_{curve} = \frac{((1 + 2Ay + B)^2)^\frac{3}{2}}{\lvert 2A \rvert}$$



In [237]:
 # Calculation of R_curve (radius of curvature)
left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

## Real world

As one of the last steps we need to convert curvation value from pixels units into meters. 
For this project we assume that the lane is about 30 meters long and 3.7 meters wide <br>

$ym\_per\_pix = \frac{30}{720}$ meters per pixel in y dimension <br>

$xm\_per\_pix = \frac{3.7}{700} $ meters per pixel in x dimension

In [ ]:
# Calculation of R_curve (radius of curvature)
left_curverad = ((1 + (2 * left_fit[0] * y_eval * ym_per_pix + left_fit[1]) ** 2) ** 1.5) / np.absolute(
    2 * left_fit[0])
right_curverad = ((1 + (2 * right_fit[0] * y_eval * ym_per_pix + right_fit[1]) ** 2) ** 1.5) / np.absolute(
    2 * right_fit[0])

Also we calculate distance from the center of the lane by taking the difference between middle point and mean value of polynomial results at y max for right and left lanes.

<figure>
 <img src="write_up_sources/test0.jpg" width=100% alt="Combined Image"  />
</figure>
 <p></p>
<figure>
 <img src="write_up_sources/test6.jpg" width=100% alt="Combined Image"  />
</figure>

## Reflection

Implemented pipeline corresponds to the proposed pipeline in the lesson. <br>
It was very intresting to compare "histogram" and "convolution" approaches. Also it is useful to know that applying threshold using different colour spaces and combine results may give much better output than just grayscale. <br>

### Weak points

Implemented pipeline requires a lot of user iteraction, such as chosing source and destination points for perspective transformation and mask region of interest. Also the curvation and deviation measurements are very approximate due to guessing the match between source and destination points for transformation. Also the algorithm is not adapted for situation changes such as change of lane form.<br>
In that case it is required to use additional sensors in order to increase precion level. <br>

## Ways to improve

Store information about previously found lane points and use them to calculate region of interest for next frame.